In [1]:
print("Hello world")

Hello world


In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'/Users/juinnshengna/Desktop/schizo/abnpsychchatbot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter #for chunking


In [5]:
#Extract text from PDF file
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf_files("data")

In [7]:
extracted_data

[Document(metadata={'producer': 'mPDF 7.0.3', 'creator': 'Washington State University', 'creationdate': "20240717221411+00'00'", 'title': 'Fundamentals of Psychological Disorders', 'subject': 'JM', 'author': 'Alexis Bridley and Lee W. Daffin Jr.', 'moddate': "20240717221411+00'00'", 'source': 'data/psych.pdf', 'total_pages': 343, 'page': 0, 'page_label': '1'}, page_content='Fundamentals of Psychological Disorders'),
 Document(metadata={'producer': 'mPDF 7.0.3', 'creator': 'Washington State University', 'creationdate': "20240717221411+00'00'", 'title': 'Fundamentals of Psychological Disorders', 'subject': 'JM', 'author': 'Alexis Bridley and Lee W. Daffin Jr.', 'moddate': "20240717221411+00'00'", 'source': 'data/psych.pdf', 'total_pages': 343, 'page': 1, 'page_label': '2'}, page_content=''),
 Document(metadata={'producer': 'mPDF 7.0.3', 'creator': 'Washington State University', 'creationdate': "20240717221411+00'00'", 'title': 'Fundamentals of Psychological Disorders', 'subject': 'JM', '

In [8]:
len(extracted_data)

343

In [9]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [11]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [12]:
minimal_docs

[Document(metadata={'source': 'data/psych.pdf'}, page_content='Fundamentals of Psychological Disorders'),
 Document(metadata={'source': 'data/psych.pdf'}, page_content=''),
 Document(metadata={'source': 'data/psych.pdf'}, page_content='Fundamentals of Psychological Disorders'),
 Document(metadata={'source': 'data/psych.pdf'}, page_content='PDF Version of the Textbook – Fundamentals of Psychological Disorders – 3rd\nedition 5TR – version 3.5\nOrder a print copy:\nhttps://www.lulu.com/shop/lee-w-daffin-jr-and-alexis-bridley/fundamentals-of-psychological-disorders/pa\nperback/product-y2kqn4.html?q=daffin&page=1&pageSize=4'),
 Document(metadata={'source': 'data/psych.pdf'}, page_content=''),
 Document(metadata={'source': 'data/psych.pdf'}, page_content='This work is licensed under a Creative Commons Attribution-NonCommercial-ShareAlike 4.0\nInternational License.\nAttribution-NonCommercial-ShareAlike 4.0 International\nOfficial translations of this license are available in other languages.

Chunking

In [13]:
#Split the documents into smaller chunks

def text_splitter(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts = text_splitter.split_documents(minimal_docs)
    return texts

In [15]:
texts_chunk = text_splitter(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 2168


In [16]:
texts_chunk

[Document(metadata={'source': 'data/psych.pdf'}, page_content='Fundamentals of Psychological Disorders'),
 Document(metadata={'source': 'data/psych.pdf'}, page_content='Fundamentals of Psychological Disorders'),
 Document(metadata={'source': 'data/psych.pdf'}, page_content='PDF Version of the Textbook – Fundamentals of Psychological Disorders – 3rd\nedition 5TR – version 3.5\nOrder a print copy:\nhttps://www.lulu.com/shop/lee-w-daffin-jr-and-alexis-bridley/fundamentals-of-psychological-disorders/pa\nperback/product-y2kqn4.html?q=daffin&page=1&pageSize=4'),
 Document(metadata={'source': 'data/psych.pdf'}, page_content='This work is licensed under a Creative Commons Attribution-NonCommercial-ShareAlike 4.0\nInternational License.\nAttribution-NonCommercial-ShareAlike 4.0 International\nOfficial translations of this license are available in other languages.\nCreative Commons Corporation (“Creative Commons”) is not a law firm and does not provide legal\nservices or legal advice. Distributi

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

/var/folders/f5/gfbd60tn7mldmd0mrjqw54sh0000gn/T/ipykernel_4556/30168171.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


README.md: 0.00B [00:00, ?B/s]

In [18]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
vector = embedding.embed_query("Hello world")
vector

[-0.03447728604078293,
 0.031023193150758743,
 0.006734972819685936,
 0.026109015569090843,
 -0.03936203196644783,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441446952521801,
 -0.047450508922338486,
 0.014758873730897903,
 0.07087536156177521,
 0.05552757531404495,
 0.019193297252058983,
 -0.026251330971717834,
 -0.01010951865464449,
 -0.026940520852804184,
 0.02230742573738098,
 -0.02222665771842003,
 -0.14969265460968018,
 -0.017493057996034622,
 0.007676220964640379,
 0.054352302104234695,
 0.0032544543500989676,
 0.03172598406672478,
 -0.08462141454219818,
 -0.029405983164906502,
 0.05159566551446915,
 0.04812401905655861,
 -0.0033147598151117563,
 -0.05827919393777847,
 0.04196927696466446,
 0.022210730239748955,
 0.128188818693161,
 -0.022338904440402985,
 -0.011656290851533413,
 0.06292837113142014,
 -0.03287629410624504,
 -0.09122607111930847,
 -0.031175294890999794,
 0.05269956961274147,
 0.04703482985496521,
 -0.08420304954051971,
 -0.030056187883019447,
 -0.02074483

In [20]:
print( "Vector length:", len(vector))

Vector length: 384


In [22]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [55]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")



In [56]:
from dotenv import load_dotenv
load_dotenv()

# Get the OpenRouter API key from .env file
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
if not openrouter_api_key:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables. Please add it to your .env file.")

In [58]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENROUTER_API_KEY"] = openrouter_api_key

In [59]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [60]:
pc

In [61]:
from pinecone import ServerlessSpec 

index_name = "abnormalpsych-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

Takes new text, converts it into vectors, and saves it into Pinecone

In [62]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

Opens and uses an index that already has data in it


In [63]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

Sample code to add more documents

dswith = Document(
    page_content="dswithbappy is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)
docsearch.add_documents(documents=[dswith])


In [64]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [65]:
retrieved_docs = retriever.invoke("What is Schizophrenia?")
retrieved_docs

[Document(id='69c3fc13-bf64-4673-8ec9-2fed6345c040', metadata={'source': 'data/psych.pdf'}, page_content='schizophrenia spectrum disorders.\nSchizophrenia is characterized by delusions, hallucinations, disorganized speech,\ndisorganized/abnormal behavior, or negative symptoms lasting six months.\nSchizophreniform disorder is considered an “intermediate” disorder between schizophrenia and\nbrief psychotic disorder as the symptoms are present for at least one month but not longer than\nsix months.\nSchizoaffective disorder is characterized by the psychotic symptoms included in schizophrenia'),
 Document(id='4993d2a6-2c2c-4764-9a4d-1489152a807b', metadata={'source': 'data/psych.pdf'}, page_content='behavior, and negative symptoms. Individuals diagnosed with a schizophrenia spectrum disorder\nexperience psychosis, which is defined as a loss of contact with reality. Psychosis episodes make it\ndifficult for individuals to perceive and respond to environmental stimuli, causing a significant\

#Refer docs -https://openrouter.ai/docs/guides/community/langchain

In [66]:
from langchain_openai import ChatOpenAI

In [79]:
llm = ChatOpenAI(
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
    model="mistralai/devstral-2512:free",
)

In [80]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [81]:

system_prompt = (
    "You are a mental health professional that helps the public to understand mental health illness and reduce stigma."
    "Use the following pieces of retrieved context to give a clear, "
    "respectful, and easy-to-understand overview. "
    "Focus on correcting common myths, explaining stigma, and promoting empathy. "
    "Avoid medical diagnosis or clinical advice. "
    "If you do not know the answer, say that you do not know. "
    "Use no more than three sentences and keep the explanation simple and concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [82]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [83]:
response = rag_chain.invoke({"input": "what is Schizophrenia?"})
print(response["answer"])

Schizophrenia is a mental health condition where a person may experience symptoms like hearing or seeing things that aren’t there (hallucinations), holding strong beliefs not based in reality (delusions), or having disorganized thoughts and behaviors. It’s important to know that schizophrenia is not a "split personality" or caused by personal weakness—it’s a real and treatable condition that affects how a person thinks, feels, and perceives the world. With support and understanding, people with schizophrenia can lead meaningful lives.


In [84]:
response = rag_chain.invoke({"input": "what is positive and negative symptoms of schizophrenia?"})
print(response["answer"])

Schizophrenia symptoms are often grouped into "positive" and "negative" categories, but these terms don't mean "good" or "bad." **Positive symptoms** are experiences *added* to a person's reality, like hallucinations (hearing/seeing things others don’t) or delusions (strongly held false beliefs). **Negative symptoms** are things *taken away* from a person’s usual functioning, like reduced emotional expression, lack of motivation, or social withdrawal. Both types of symptoms can be challenging, but negative symptoms often last longer and can be harder to treat.


In [85]:
response = rag_chain.invoke({"input": "What is the DSM criteria for ADHD?"})
print(response["answer"])

The DSM-5-TR outlines that ADHD symptoms are generally organized into two main categories: hyperactivity and impulsivity symptoms, and inattention symptoms. It's important to note that ADHD is a neurodevelopmental disorder, meaning it begins in childhood and can continue into adulthood. However, I don't have the specific criteria details here, so I recommend checking the DSM-5-TR or consulting a mental health professional for more information.


In [86]:
response = rag_chain.invoke({"input": "How do I differentiate MDD and SAD and Persistent Depressive Disorder?"})
print(response["answer"])

I’m glad you’re seeking to understand these conditions better! Here’s a simple breakdown:

- **Major Depressive Disorder (MDD)** involves intense episodes of depression (like deep sadness or loss of interest) lasting at least two weeks, but these episodes may come and go.
- **Persistent Depressive Disorder (PDD)** is a milder but long-lasting form of depression—symptoms (like low mood or fatigue) continue for *at least two years* without significant breaks.
- **Seasonal Affective Disorder (SAD)** is a type of depression tied to seasonal changes (usually winter), with symptoms like low energy or sadness appearing and fading with the seasons.

**Key difference:** MDD is episodic, PDD is chronic, and SAD is seasonal. All are valid experiences, and none are "just sadness"—they’re real challenges that deserve compassion.

*(Note: If you’re supporting someone, focus on listening and validating their feelings rather than labeling their experience.)*
